I need help writing this code.

The salaries csv, when asked to return the draft players, only returns about 45/60 players.
Why? because the remaining 15 who are not in the csv are currently overseas.
Essentially, if the player is currently not in the league, then they are not in the "salaries csv"

Solution:
The nba_api has all the players from the draft every year (60/60).
So bridge the nba_api with the csv's using the draft info (year and overall pick).
Then fill in 0's for players who just aren't in the league anymore (example: jeremy lin)

Since there is so much code here that is hard to read, I've made a new file called "GetSalaryFromCSV and API.ipynb".
That's where I feel like it would be best to work on this problem


In [1]:
import pandas as pd
import numpy as np
salaries = pd.read_csv('/Users/josephhartono/Downloads/salaries_1985to2018.csv')
players = pd.read_csv('/Users/josephhartono/Downloads/players.csv')
merged = pd.merge(left=salaries, right=players, left_on='player_id', right_on='_id')

In [2]:
draft_year = 2014

In [3]:
#this function gets the draft picks from a given year
def getDraft(leagueID, year): #"00, 01, 10, 20", "string(2015)" ###api is string
    from nba_api.stats.endpoints import drafthistory
    draftHist = drafthistory.DraftHistory(leagueID)
    draftHistDF = draftHist.get_data_frames()[0]
    yearDraft = draftHistDF[(draftHistDF['SEASON']==year)]
    return yearDraft
a2017 = getDraft("00", str(2017))
#a2017

In [4]:
#this function outputs a list with ordinal values ranging from 1 - 60
def getString_range(rangeLength):
    list_of_lists = []
    ordinalList = []
    rangeVal = rangeLength

    for n in range(1,rangeVal):
        if(n == 11):
            ordinalList.append(str(n) + "th overall")
        elif(n == 12):
            ordinalList.append(str(n) + "th overall")
        elif(n == 13):
            ordinalList.append(str(n) + "th overall")        
        elif((n % 10) == 1):
            ordinalList.append(str(n) + "st overall")
        elif((n % 10) == 2):
            ordinalList.append(str(n) + "nd overall")
        elif((n % 10) == 3):
            ordinalList.append(str(n) + "rd overall")
        else:
            ordinalList.append(str(n) + "th overall")
            
    list_of_lists.append(ordinalList)
    #print(ordinalList)

    rangeList = []
    for n in range(1, rangeVal):
        rangeList.append(str(n))
    list_of_lists.append(rangeList)

    #print(rangeList)
    return list_of_lists


In [1]:
#this function gets all players who were drafted in a given year and replaces the ordinal values with the values we want
def convertSalaryCSV_ordinal(): #this is the csv that is incomplete, this is not the data that I want
    Draft = players.loc[(players['draft_year']== str(draft_year + 4))].copy()
    list = getString_range(61)
    Draft['draft_pick'] = testDraft['draft_pick'].replace(list[0], list[1])
    Draft['draft_pick'] = testDraft['draft_pick'].astype(int)
    Draft.sort_values(by='draft_pick', ascending=True)
    Draft
    return Draft
#convertSalaryCSV_ordinal()

In [31]:
def getSalaryForYear(year): #input is int
    return salaries.loc[(salaries['season_start']==year)].copy() #not a string input ### csv is a int input

In [8]:
#this function gets all the players in a given year from the nba_api
def getPlayers(year):
    from nba_api.stats.endpoints import commonallplayers
    allPlayers = commonallplayers.CommonAllPlayers(is_only_current_season=0, league_id='00').get_data_frames()
    allPlayers = allPlayers[0]
    allPlayersThatYear = allPlayers[allPlayers['TO_YEAR'] >= str(year)]
    allPlayersThatYear = allPlayersThatYear[allPlayersThatYear['FROM_YEAR'] <= str(year)]
    return allPlayersThatYear


def getCareerFrom_ID_and_season(ID, season_year, seasonType = "Regular Season"): #string, string
    from nba_api.stats.endpoints import playergamelog
    career_this_season = playergamelog.PlayerGameLog(ID, season_year, 'Regular Season')
    #print(type(career_this_season))
    return career_this_season.get_data_frames()[0]

In [23]:
players2014 = getPlayers('2014')

def getIDfromName(name):
    from nba_api.stats.static import players
    current = players.find_players_by_full_name(name)
    return current[0]["id"]
ID = getIDfromName("Steven Adams")
#ID

id_Dataframe = pd.DataFrame().assign(id=players2014['PERSON_ID'])


def getDraftInfo(data):
    from nba_api.stats.endpoints import commonplayerinfo
    overall_pick = []
    draft_year = []
    

players2014


,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
15,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,quincy_acy,0,,,,,None,Y,01
20,203919,"Adams, Jordan",Jordan Adams,0,2014,2015,jordan_adams,jordan_adams,0,,,,,None,Y,01
22,203500,"Adams, Steven",Steven Adams,1,2013,2021,steven_adams,steven_adams,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,00
27,202399,"Adrien, Jeff",Jeff Adrien,0,2010,2014,jeff_adrien,jeff_adrien,0,,,,,None,Y,01
28,201167,"Afflalo, Arron",Arron Afflalo,0,2007,2017,arron_afflalo,arron_afflalo,0,,,,,None,Y,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,203923,"Young, James",James Young,0,2014,2017,james_young,james_young,0,,,,,None,Y,01
4704,201156,"Young, Nick",Nick Young,0,2007,2018,nick_young,nick_young,0,,,,,None,Y,00
4707,201152,"Young, Thaddeus",Thaddeus Young,1,2007,2021,thaddeus_young,thaddeus_young,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
4713,203469,"Zeller, Cody",Cody Zeller,0,2013,2021,cody_zeller,cody_zeller,0,,,,,None,Y,00


get draft pick salaries at their 5th season -> put the salaries into catagories (max, close to max, average, below average, min)

put salaries of draft picks into bins

After 4 years, to see how many draft picks are getting the higher contracts

look at all salaries in 2017, get highest salary, any salary within 5 mil of that salary is considered high

2013 -> draft, get salaries -> bar graph of 2017 data

2017 ->get all player salaries -> numpy array, of type int, get highest value of array, get average of array (avg contract), lowest salary, compute close to max, 
    now make a bar graph based on the max, avg,
End goal: use 2013 data to put into 2017 data


In [19]:
#This just selects a column in a dataframe and puts it into a list...salaries csv
# Create New DataFrame of Specific column by DataFrame.assign() method.
df2 = pd.DataFrame().assign(playerID=year_salary['player_id'])
#df2
#print(df2)
df2 = df2.reset_index()
names = []
for index, row in df2.iterrows():
    indi = players.loc[players['_id'] == row['playerID']]
    #print(indi)
    names.append(indi.iloc[0]['name'])



In [39]:
merged

,league,player_id,salary,season,season_end,season_start,team,_id,birthDate,birthPlace,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers,abdelal01,"June 24, 1968","Cairo, Egypt",...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers,abdelal01,"June 24, 1968","Cairo, Egypt",...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics,abdelal01,"June 24, 1968","Cairo, Egypt",...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics,abdelal01,"June 24, 1968","Cairo, Egypt",...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings,abdelal01,"June 24, 1968","Cairo, Egypt",...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14158,NBA,zipsepa01,750000,2016-17,2017,2016,Chicago Bulls,zipsepa01,"February 18, 1994","Heidelberg, Germany",...,48th overall,2nd round,Chicago Bulls,2016,6-8,NaN,Paul Zipser,Small Forward,Right,215lb
14159,NBA,zipsepa01,1312611,2017-18,2018,2017,Chicago Bulls,zipsepa01,"February 18, 1994","Heidelberg, Germany",...,48th overall,2nd round,Chicago Bulls,2016,6-8,NaN,Paul Zipser,Small Forward,Right,215lb
14160,NBA,zizican01,1645200,2017-18,2018,2017,Cleveland Cavaliers,zizican01,"January 4, 1997","Split, Croatia",...,23rd overall,1st round,Boston Celtics,2016,6-11,NaN,Ante Zizic,Center,Right,254lb
14161,NBA,zubaciv01,1034956,2016-17,2017,2016,Los Angeles Lakers,zubaciv01,"March 18, 1997","Mostar, Bosnia and Herzegovina",...,32nd overall,2nd round,Los Angeles Lakers,2016,7-1,NaN,Ivica Zubac,Center,Right,240lb


In [35]:
s2015 = getSalaryForYear(2015)
s2015

,league,player_id,salary,season,season_end,season_start,team
52,NBA,acyqu01,981348,2015-16,2016,2015,Sacramento Kings
62,NBA,adamsjo01,1404600,2015-16,2016,2015,Memphis Grizzlies
74,NBA,adamsst01,2279040,2015-16,2016,2015,Oklahoma City Thunder
96,NBA,afflaar01,8000000,2015-16,2016,2015,New York Knicks
127,NBA,ajincal01,4300000,2015-16,2016,2015,New Orleans Pelicans
...,...,...,...,...,...,...,...
14105,NBA,youngjo01,1000000,2015-16,2016,2015,Indiana Pacers
14117,NBA,youngni01,5219169,2015-16,2016,2015,Los Angeles Lakers
14132,NBA,youngth01,11200000,2015-16,2016,2015,Brooklyn Nets
14139,NBA,zelleco01,4204200,2015-16,2016,2015,Charlotte Hornets


Salaries csv -> dataframe ->get draftInfo and Salaries

CommonPlayerInfo gets the round draft info from ID
I want to get draft info
and connect it with salaries

steps:
1. get players for year
2. for every player, use that ID and get the draft info
3. use draft info to get salaries
4. add those salaries to a copied dataframe with another column

NOW I have a dataframe of the salaries of every year from



In [ ]:
subsetDataFrame = dfObj[dfObj['Product'].isin(['Mangos', 'Grapes']) ]
